### Fitting DCC GARCH model using R - switch kernel

In [3]:
#install.packages("rmgarch")
suppressMessages(library(rmgarch))
suppressMessages(library(parallel))
suppressMessages(library(quantmod))
suppressMessages(library(tidyverse))
suppressMessages(library(xts))
suppressMessages(library(ggplot2))
suppressMessages(library(reshape2))

In [4]:
etfs <- read.csv("../data/etfs.csv", sep=";")

In [5]:
#etfs <- etfs %>% select(-Date)
etfs <- etfs %>% mutate(Date=as.Date(Date, format = "%Y-%m-%d"))

### Converting dataset to xts

In [6]:
xts_etfs <- xts(etfs, order.by = etfs[,1])

In [8]:
xspec = ugarchspec(mean.model = list(armaOrder = c(0, 0)), variance.model = list(garchOrder = c(1,1), model = 'gjrGARCH'), distribution.model = 'ged')
uspec = multispec(replicate(10, xspec))
spec1 = dccspec(uspec = uspec, dccOrder = c(1, 1), distribution = 'mvnorm')
cl = makePSOCKcluster(4)
multf = multifit(uspec, xts_etfs[,2:11], cluster = cl)
fit1 = dccfit(spec1, data = xts_etfs[,2:11], fit.control = list(eval.se = TRUE), fit = multf, cluster = cl)
print(fit1)           
stopCluster(cl)


*---------------------------------*
*          DCC GARCH Fit          *
*---------------------------------*

Distribution         :  mvnorm
Model                :  DCC(1,1)
No. Parameters       :  107
[VAR GARCH DCC UncQ] : [0+60+2+45]
No. Series           :  10
No. Obs.             :  2529
Log-Likelihood       :  -29988.88
Av.Log-Likelihood    :  -11.86 

Optimal Parameters
-----------------------------------
              Estimate  Std. Error    t value Pr(>|t|)
[EEM].mu      0.016392    0.022383   0.732329 0.463968
[EEM].omega   0.043291    0.015727   2.752643 0.005912
[EEM].alpha1  0.031917    0.021794   1.464516 0.143053
[EEM].beta1   0.884587    0.029671  29.813201 0.000000
[EEM].gamma1  0.111623    0.022717   4.913598 0.000001
[EEM].shape   1.688241    0.077598  21.756218 0.000000
[EWZ].mu      0.002607    0.034561   0.075418 0.939882
[EWZ].omega   0.076696    0.033699   2.275899 0.022852
[EWZ].alpha1  0.042215    0.016926   2.494092 0.012628
[EWZ].beta1   0.901674    0.024340 

In [11]:
coef(fit1)

                 EEM      EWZ      FXI      GDX       HYG      SLV       SPY
2021-09-13 1.0105063 2.464456 1.487492 1.855005 0.2187609 1.552429 0.6578703
2021-09-14 0.9639216 2.431469 1.422294 1.868405 0.2322735 1.504888 0.6100608
2021-09-15 0.9675328 2.325922 1.528542 1.817387 0.2214658 1.465849 0.6368146
2021-09-16 0.9224669 2.213234 1.474223 1.764029 0.2192736 1.422662 0.6827318
2021-09-17 0.9708540 2.165357 1.479594 2.018712 0.2100608 1.667084 0.6355309
2021-09-20 0.9389232 2.233906 1.424232 1.976229 0.2088227 1.722008 0.8677762
                 TLT       XLF       XLU
2021-09-13 0.7892839 0.8494309 0.9936961
2021-09-14 0.7776482 0.9190648 0.9532933
2021-09-15 0.8214167 1.0513886 0.9199914
2021-09-16 0.7955305 1.0357515 0.8839057
2021-09-17 0.7766354 0.9675136 0.8857085
2021-09-20 0.7607383 0.9265005 0.9645243

In [35]:
#coef(fit1) %>% write.csv('coef.csv')
#install.packages("gmvarkit")
library(gmvarkit)

In [29]:
etf_forecast <- dccforecast(fit1, n.ahead=1)

In [39]:
rcov(etf_forecast, output="matrix")

,EEM,EWZ:EEM,FXI:EEM,GDX:EEM,HYG:EEM,SLV:EEM,SPY:EEM,TLT:EEM,XLF:EEM,XLU:EEM,⋯,SPY,TLT:SPY,XLF:SPY,XLU:SPY,TLT,XLF:TLT,XLU:TLT,XLF,XLU:XLF,XLU
T+1,0.008088057,-6.194942e-05,0.01780162,0.01094251,0.02511785,0.002839959,0.08531867,0.03429511,0.08243349,0.08448728,⋯,0.9132665,0.3688015,0.8818356,0.9057751,0.1500154,0.3559758,0.3666705,0.8522604,0.8746033,0.8998687


In [50]:
rcov(etf_forecast, output="matrix")

,EEM,EWZ:EEM,FXI:EEM,GDX:EEM,HYG:EEM,SLV:EEM,SPY:EEM,TLT:EEM,XLF:EEM,XLU:EEM,⋯,SPY,TLT:SPY,XLF:SPY,XLU:SPY,TLT,XLF:TLT,XLU:TLT,XLF,XLU:XLF,XLU
T+1,0.008088057,-6.194942e-05,0.01780162,0.01094251,0.02511785,0.002839959,0.08531867,0.03429511,0.08243349,0.08448728,⋯,0.9132665,0.3688015,0.8818356,0.9057751,0.1500154,0.3559758,0.3666705,0.8522604,0.8746033,0.8998687


In [ ]:
etf_forecast

### Using DCC-roll to forecast

In [104]:
cl = makePSOCKcluster(8)
roll <- dccroll(spec = spec1, data = xts_etfs[,2:11], forecast.length = 1, n.ahead=1, refit.every = 1,
               refit.window = "recursive")

#,             fit.control = list(eval.se = TRUE, stationarity = TRUE, scale = FALSE))
stopCluster(cl)

Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”
Warning message:
“unidentified option(s) in fit.control:
 eval.se”


In [99]:
data <- as_tibble(rcov(roll, output="matrix")) %>%
        mutate(Date = seq.int(1, nrow(rcov(roll, output="matrix")), 1), .before=EEM) 

print(data)

# A tibble: 25 × 56
    Date   EEM `EWZ:EEM` `FXI:EEM` `GDX:EEM` `HYG:EEM` `SLV:EEM` `SPY:EEM`
   <dbl> <dbl>     <dbl>     <dbl>     <dbl>     <dbl>     <dbl>     <dbl>
 1     1 1.62       1.91      2.53     0.670     0.217     0.619     0.524
 2     2 1.46       1.73      2.30     0.615     0.195     0.567     0.492
 3     3 1.29       1.57      2.06     0.581     0.179     0.529     0.457
 4     4 1.25       1.45      1.94     0.642     0.167     0.624     0.412
 5     5 1.12       1.31      1.81     0.610     0.156     0.627     0.366
 6     6 1.01       1.19      1.64     0.566     0.146     0.574     0.327
 7     7 0.917      1.08      1.49     0.548     0.135     0.538     0.299
 8     8 0.935      1.12      1.48     0.565     0.129     0.555     0.289
 9     9 0.859      1.02      1.35     0.534     0.122     0.529     0.267
10    10 0.893      1.08      1.37     0.532     0.116     0.510     0.262
# … with 15 more rows, and 48 more variables: TLT:EEM <dbl>, XLF:EEM <dbl>,
#   

In [ ]:
data %>% ggplot(aes(x = Date)) +
geom_line(color = Series) +
# Add a second axis and specify its features
labs(
  title="Macroeconomic indicators",
  x = "Year",
  y = "Count")


#rcov(roll, output="matrix")